In [3]:
# Import libraries and modules
import csv  # for reading and writing CSV files
import matplotlib.pyplot as plt  # for creating plots and charts
import numpy as np  # for numerical computing with Python
import pandas as pd  # for data manipulation and analysis
import pathlib  # for interacting with file paths in a cross-platform manner
import tensorflow as tf  # for machine learning and deep learning
from keras.layers import Dense, Activation  # for building deep learning models in TensorFlow
from keras.models import Sequential  # for building deep learning models in TensorFlow
from keras.optimizers import Adam, RMSprop  # for building deep learning models in TensorFlow
from keras.callbacks import History
from sklearn.model_selection import train_test_split  # for model selection and evaluation
from sklearn.preprocessing import MinMaxScaler  # for preprocessing data
from tensorflow import keras  # for building deep learning models in TensorFlow
from tensorflow.keras import layers  # for building deep learning models in TensorFlow
from keras.layers import Conv1D, MaxPooling1D


# Print TensorFlow version
print(tf.__version__)

# Define column names for the CSV file
column_names = ['Open Time','Open','High','Low','Close',
                'Close Volume', 'Time', 'Quote asset volume',
               'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
               

# Read in the CSV file using pandas
raw_dataset = pd.read_csv("main.csv", names=column_names,
                      na_values = "?", comment='\t', dtype='float',
                      sep=",", skipinitialspace=True)

# Copy the data from the raw dataset to a new dataframe
dataset = raw_dataset.copy()

# Print the last few rows of the dataset
dataset.tail()


2023-01-07 21:54:44.806261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 21:54:47.552775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-07 21:54:47.552925: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-07 21:54:47.552934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

2.11.0


,Open Time,Open,High,Low,Close,Close Volume,Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
188312,1.620794e+12,57517.42,57526.28,57485.00,57485.07,42.575735,1.620794e+12,2.448258e+06,1195.0,15.319691,880913.090845
188313,1.620794e+12,57485.07,57496.42,57466.75,57481.49,34.205467,1.620794e+12,1.966194e+06,1096.0,15.971891,918058.816162
188314,1.620794e+12,57477.18,57509.99,57458.18,57470.00,30.211789,1.620794e+12,1.736514e+06,955.0,13.054229,750364.577319
188315,1.620794e+12,57470.00,57470.01,57400.00,57450.90,45.354728,1.620794e+12,2.605080e+06,1559.0,12.615628,724559.233035
188316,1.620794e+12,57450.89,57475.66,57435.51,57450.19,14.168318,1.620794e+12,8.140594e+05,730.0,7.247751,416412.022206


In [4]:
dataset.pop("Open Time")
dataset.pop("Time")
# Randomly select 80% of the rows from the dataset and store them in a new dataframe
train_dataset = dataset.sample(frac=0.8,random_state=0)

# Remove the rows in the training dataset from the original dataset, leaving the remaining rows in a new dataframe
test_dataset = dataset.drop(train_dataset.index)


In [5]:
# Calculate statistical summary of the training dataset
train_stats = train_dataset.describe()

# Remove the "Close" column from the statistical summary
train_stats.pop("Close")

# Transpose the statistical summary so that it's in a more useful shape
train_stats = train_stats.transpose()

In [6]:
# Remove the "Close" column from the training dataset and store it in a new dataframe
train_labels = train_dataset.pop('Close')

# Remove the "Close" column from the testing dataset and store it in a new dataframe
test_labels = test_dataset.pop('Close')

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_dataset)
normed_train_dataset = scaler.transform(train_dataset)
normed_test_dataset = scaler.transform(test_dataset)

In [8]:
print(normed_train_dataset.shape)
print(train_labels.shape)

(150654, 8)
(150654,)


In [9]:
print(train_labels)

54176     38273.80
22054     36653.35
25894     36570.67
3604      34482.99
96019     50880.06
            ...   
58124     46408.25
114876    57751.49
89163     51009.62
155465    56929.00
90475     48084.80
Name: Close, Length: 150654, dtype: float64


In [18]:
#=============== Parte Manual ===============#
neuronios = 32
act_h = "relu"
act_out = "relu"
learnr = 0.001
camadas = 1
epochs_n = 30

# Set up neural network model
"""
model = keras.Sequential([
    layers.Dense(neuronios, activation=act_h, input_shape=[8]),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    #layers.Dense(neuronios, activation=act_h),
    #layers.Dense(neuronios, activation=act_h),
    layers.Dense(1,activation=act_out)
  ])
"""

# Set up model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(8, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(Dense(1, activation='relu'))


# Compile model
optimizer = Adam(learnr)

model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])

In [19]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 6, 32)             128       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 3, 32)            0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 96)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 97        
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [20]:
example_batch = normed_train_dataset[:8]
example_result = model.predict(example_batch)
example_result

1/1 [==============================] - 0s 121ms/step


array([[0.00070898],
       [0.00113801],
       [0.        ],
       [0.00382435],
       [0.        ],
       [0.00504166],
       [0.        ],
       [0.00436885]], dtype=float32)

In [21]:
# Set up callbacks
history = History()

# Train model
model.fit(normed_train_dataset, train_labels, verbose=2, batch_size=5, epochs= epochs_n, callbacks=[history])

Epoch 1/30
30131/30131 - 106s - loss: 696409792.0000 - mae: 18250.3242 - mse: 696409792.0000 - 106s/epoch - 4ms/step
Epoch 2/30
30131/30131 - 100s - loss: 1434374.7500 - mae: 667.2150 - mse: 1434374.7500 - 100s/epoch - 3ms/step
Epoch 3/30
30131/30131 - 94s - loss: 918578.2500 - mae: 527.1177 - mse: 918578.2500 - 94s/epoch - 3ms/step
Epoch 4/30
30131/30131 - 95s - loss: 551395.8125 - mae: 402.2287 - mse: 551395.8125 - 95s/epoch - 3ms/step
Epoch 5/30
30131/30131 - 102s - loss: 307267.8750 - mae: 289.8564 - mse: 307267.8750 - 102s/epoch - 3ms/step
Epoch 6/30
30131/30131 - 116s - loss: 149282.9844 - mae: 187.1277 - mse: 149282.9844 - 116s/epoch - 4ms/step
Epoch 7/30
30131/30131 - 103s - loss: 62852.9062 - mae: 102.4128 - mse: 62852.9062 - 103s/epoch - 3ms/step
Epoch 8/30
30131/30131 - 103s - loss: 27288.9648 - mae: 54.6871 - mse: 27288.9648 - 103s/epoch - 3ms/step
Epoch 9/30
30131/30131 - 104s - loss: 18808.6367 - mae: 47.4876 - mse: 18808.6367 - 104s/epoch - 3ms/step
Epoch 10/30
30131/301

In [22]:
# Evaluate model
results = model.evaluate(normed_test_dataset, test_labels)
print("Resultado",results)

# Print average loss over all epochs
print("Average loss:", np.mean(history.history['mse']))

1177/1177 [==============================] - 4s 3ms/step - loss: 4505.3271 - mae: 32.8007 - mse: 4505.3271
Resultado [4505.3271484375, 32.80067825317383, 4505.3271484375]
Average loss: 23334678.9888265


In [23]:
# predictions
y_pred = model.predict(normed_test_dataset)
print(y_pred)

1177/1177 [==============================] - 3s 3ms/step
[[28987.543]
 [28857.324]
 [28751.812]
 ...
 [57846.402]
 [57699.83 ]
 [57491.797]]


In [24]:
filename = 'modelmanualcnn_c'+ str(camadas) +'_n'+ str(neuronios) +'_actn'+str(act_h) +'_acto'+str(act_out) +'_lr' + str(learnr) + '_lossmae' +'_ep' + str(epochs_n)
model.save(filename, save_format='tf')

INFO:tensorflow:Assets written to: modelmanualcnn_c4_n32_actnrelu_actosigmoid_lr0.001_lossmae_ep30/assets


INFO:tensorflow:Assets written to: modelmanualcnn_c4_n32_actnrelu_actosigmoid_lr0.001_lossmae_ep30/assets
